In [1]:
import gym
import pybullet_envs
import numpy as np
#from sac_agent import Agent
import matplotlib.pyplot as plt
from datetime import datetime
from hidio_agent import Agent
import os

C:\Users\areeb\anaconda3\envs\mscHRL\lib\site-packages\gym\envs\registration.py:441: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  "The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead."


In [9]:
import os
os.path.isfile("../network_checkpoints/sac/InvertedPendulumBulletEnv-v0/actor_network.pth")

True

In [6]:
os.getcwd()

'C:\\Users\\areeb\\Documents\\MSc Artificial Intelligence & Machine Learning\\Summer Project\\Potential supervisors\\Mohan Sridharan\\project\\mscHRL\\hidio\\hidio_scratch\\hidio_and_sac'

In [2]:
env = gym.make("InvertedPendulumBulletEnv-v0")
#gym.make("HalfCheetahBulletEnv-v0")
#gym.make("BipedalWalker-v3")
#gym.make("HumanoidBulletEnv-v0")
#gym.make("HopperBulletEnv-v0")
#gym.make("InvertedPendulumBulletEnv-v0")
checkpoint_dir = "../network_checkpoints/hidio/"
option_interval = 3
batch_size = 500
num_games = 5
#550
best_score = env.reward_range[0]
score_history = []
worker_score_history = []
load_checkpoint = True
transfer_network_params = False
transfer_network_path = "./"
env_name = env.spec.id


agent = Agent(env=env, 
              skill_dims=5, 
              learning_rate=3*10**-4, 
              memory_size=10**6, 
              checkpoint_dir=checkpoint_dir, 
              option_interval=option_interval, 
              gamma=0.99, 
              option_gamma=0.99, 
              reward_scale=2, 
              batch_size=batch_size, 
              polyak_coeff=0.995, 
              beta=0.01, 
              alpha=0.35, 
              use_auto_entropy_adjustment=True,
              w_alpha=0.2, 
              w_auto_entropy_adjustment=False, 
              use_tanh=True, 
              feature="stateAction")


if load_checkpoint:
    agent.load_models()
    #env.render(mode="human")

#if transfer_network_params:
#    agent.transfer_network_params(load_path=transfer_network_path)

for j in range(num_games):
    observation = env.reset()
    done = False
    score = 0
    worker_score = 0

    frame2 = env.render(mode="rgb_array")

    while not done:
        skill = agent.generate_skill(state=observation)
        states_array, actions_array, next_states_array, rewards_array, actor_log_probs_array, done_array = agent.generate_empty_arrays()

        for i in range(option_interval):
            action, actor_log_probs = agent.worker.choose_action(state=observation, skill=skill)
            next_observation, reward, done, info = env.step(action)
            states_array[i, :] = observation
            actions_array[i, :] = action
            next_states_array[i, :] = next_observation
            rewards_array[i] = reward
            actor_log_probs_array[i] = actor_log_probs.item()
            done_array[i] = done

        agent.worker.remember(state_array=states_array, 
                              action_array=actions_array, 
                              next_state_array=next_states_array, 
                              skill=skill, 
                              reward_array=rewards_array, 
                              done_array=done_array)
        scheduler_reward = agent.post_interval_reward(actor_log_probs=actor_log_probs_array, 
                                                      reward_array=rewards_array, 
                                                      expected_log_probs=False)
        agent.remember(state=observation, 
                       skill=skill, 
                       next_state=next_observation, 
                       reward=scheduler_reward, 
                       done=done)

        score = score + scheduler_reward

        if not load_checkpoint:
            if agent.scheduler_memory.memory_counter > batch_size:
                worker_reward = agent.learn()
                worker_score = worker_score + worker_reward.detach().cpu().numpy()
            else:
                agent.learn()

        observation = next_observation


    score_history.append(score)
    avg_score = np.mean(score_history[-20:])

    if avg_score > best_score:
        best_score = avg_score
        if not load_checkpoint:
            agent.save_models()

    if not load_checkpoint:
        worker_score_history.append(worker_score)
        avg_worker_score = np.mean(worker_score_history[-100:])
        print("Episode = {}, Score = {:.1f}, Average score = {:.2f}, Worker score = {:.3f}, Average worker score = {:.3f}".format(j, score, avg_score, worker_score, avg_worker_score))

    else:
        print("Episode = {}, Score = {:.1f}, Average score = {:.2f}".format(j, score, avg_score))

    print(agent.alpha)
    
    plt.imshow(frame2)
    plt.show()


C:\Users\areeb\anaconda3\envs\mscHRL\lib\site-packages\gym\core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
C:\Users\areeb\anaconda3\envs\mscHRL\lib\site-packages\gym\wrappers\step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
C:\Users\a

************--Loading scheduler--************


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [1]:
import torch as T
T.cuda.is_available()

False